# プロンプト／レスポンス最適化
* 『LangChainとLangGraphによるRAG・AIエージェント実践入門（12章より)』
* 生成された目標やユーザー要求を、より効果的なプロンプトに変換しLLMからより質の高い回答を得るパターン


In [2]:
# !pip install langchain-core==0.3.0 langchain-community==0.3.0 \
# langgraph==0.2.22 langchain-openai==0.2.0 langchain-anthropic==0.2.0 \
# numpy==1.26.4 faiss-cpu==1.8.0.post1 \
# pydantic-settings==2.5.2 retry==0.9.2 decorator==4.4.2

In [3]:
# !pip install "pydantic<2.11"  # 例: 2.10.6 エラー回避用

In [23]:
from dotenv import load_dotenv
load_dotenv()

In [24]:
# import os

# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
# os.environ["LANGCHAIN_PROJECT"] = LANGCHAIN_PROJECT

In [25]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from agent_tools.passive_goal_creator.main import Goal, PassiveGoalCreator
from pydantic import BaseModel, Field
from pprint import pprint

# プロンプト最適化
* 入力された目標を基に、効果的なプロンプトを生成するプログラム

## データモデル定義
* ここでの最適化の定義「より具体的で測定可能なもの」
* 新たにmetricsフィールドを追加し、「目標の達成度を測定する方法」が設定されるように指示している。これにより、各目標に対して必ずその達成度を測定するための測定方法が含まれることになる

In [26]:
class OptimizedGoal(BaseModel):
    description: str = Field(..., description="目標の説明")
    metrics: str = Field(..., description="目標の達成度を測定する方法")

    @property
    def text(self) -> str:
        return f"{self.description}(測定基準: {self.metrics})"

## プロンプトとチェーン
* 各目標を具体的、測定可能、達成可能、関連性が高いものに最適化するように指示

In [27]:
class PromptOptimizer:
    def __init__(self, llm: ChatOpenAI):
        self.llm = llm

    def run(self, query: str) -> OptimizedGoal:
        prompt = ChatPromptTemplate.from_template(
            "あなたは目標設定の専門家です。以下の目標をSMART原則（Specific: 具体的、Measurable: 測定可能、Achievable: 達成可能、Relevant: 関連性が高い、Time-bound: 期限がある）に基づいて最適化してください。\n\n"
            "元の目標:\n"
            "{query}\n\n"
            "指示:\n"
            "1. 元の目標を分析し、不足している要素や改善点を特定してください。\n"
            "2. あなたが実行可能な行動は以下の行動だけです。\n"
            "   - インターネットを利用して、目標を達成するための調査を行う。\n"
            "   - ユーザーのためのレポートを生成する。\n"
            "3. SMART原則の各要素を考慮しながら、目標を具体的かつ詳細に記載してください。\n"
            "   - 一切抽象的な表現を含んではいけません。\n"
            "   - 必ず全ての単語が実行可能かつ具体的であることを確認してください。\n"
            "4. 目標の達成度を測定する方法を具体的かつ詳細に記載してください。\n"
            "5. 元の目標で期限が指定されていない場合は、期限を考慮する必要はありません。\n"
            "6. REMEMBER: 決して2.以外の行動を取ってはいけません。"
        )
        chain = prompt | self.llm.with_structured_output(OptimizedGoal)
        return chain.invoke({"query": query})

## 実行

In [21]:
query = "カレーライスの作り方"

llm = ChatOpenAI(model="gpt-4o", temperature=0.0)

# Passive Goal Creatorで目標を具体化
passive_goal_creator = PassiveGoalCreator(llm=llm)
goal: Goal = passive_goal_creator.run(query=query)

# Prompt Optimizerによるプロンプト最適化
prompt_optimizer = PromptOptimizer(llm=llm)
optimized_goal: OptimizedGoal = prompt_optimizer.run(query=goal.text)

display(f"Passive Goal Creator出力：{goal.text}")
print("\nプロンプト最適化結果：\n")
display(f"{optimized_goal.text}")

'Passive Goal Creator出力：カレーライスの作り方を調査し、ユーザーにわかりやすいレポートを生成する。レポートには必要な材料、調理手順、調理時間、コツや注意点を含める。'


プロンプト最適化結果：



'インターネットを利用して、カレーライスの作り方に関する情報を調査し、ユーザーにわかりやすいレポートを生成する。レポートには以下の要素を含める: 必要な材料のリスト、各材料の分量、調理手順の詳細なステップバイステップガイド、調理時間の見積もり、調理のコツや注意点。(測定基準: 生成したレポートが以下の基準を満たしているかを確認する: 1) 必要な材料が全てリストアップされていること。2) 各材料の分量が明記されていること。3) 調理手順がステップバイステップで詳細に記載されていること。4) 調理時間が具体的に見積もられていること。5) 調理のコツや注意点が少なくとも3つ含まれていること。)'

# レスポンス最適化
* ユーザーが入力したプロンプトから、そのプロンプトの目標を達成するために必要なレスポンス仕様を出力するプロンプトを定義

In [28]:
class ResponseOptimizer:
    def __init__(self, llm: ChatOpenAI):
        self.llm = llm

    def run(self, query: str) -> str:
        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "あなたはAIエージェントシステムのレスポンス最適化スペシャリストです。与えられた目標に対して、エージェントが目標にあったレスポンスを返すためのレスポンス仕様を策定してください。",
                ),
                (
                    "human",
                    "以下の手順に従って、レスポンス最適化プロンプトを作成してください：\n\n"
                    "1. 目標分析:\n"
                    "提示された目標を分析し、主要な要素や意図を特定してください。\n\n"
                    "2. レスポンス仕様の策定:\n"
                    "目標達成のための最適なレスポンス仕様を考案してください。トーン、構造、内容の焦点などを考慮に入れてください。\n\n"
                    "3. 具体的な指示の作成:\n"
                    "事前に収集された情報から、ユーザーの期待に沿ったレスポンスをするために必要な、AIエージェントに対する明確で実行可能な指示を作成してください。あなたの指示によってAIエージェントが実行可能なのは、既に調査済みの結果をまとめることだけです。インターネットへのアクセスはできません。\n\n"
                    "4. 例の提供:\n"
                    "可能であれば、目標に沿ったレスポンスの例を1つ以上含めてください。\n\n"
                    "5. 評価基準の設定:\n"
                    "レスポンスの効果を測定するための基準を定義してください。\n\n"
                    "以下の構造でレスポンス最適化プロンプトを出力してください:\n\n"
                    "目標分析:\n"
                    "[ここに目標の分析結果を記入]\n\n"
                    "レスポンス仕様:\n"
                    "[ここに策定されたレスポンス仕様を記入]\n\n"
                    "AIエージェントへの指示:\n"
                    "[ここにAIエージェントへの具体的な指示を記入]\n\n"
                    "レスポンス例:\n"
                    "[ここにレスポンス例を記入]\n\n"
                    "評価基準:\n"
                    "[ここに評価基準を記入]\n\n"
                    "では、以下の目標に対するレスポンス最適化プロンプトを作成してください:\n"
                    "{query}",
                ),
            ]
        )
        chain = prompt | self.llm | StrOutputParser()
        return chain.invoke({"query": query})

## 実行

In [32]:
query = "カレーライスの作り方"

llm = ChatOpenAI(model="gpt-4o", temperature=0.0)

# Passive Goal Creatorで目標を具体化
passive_goal_creator = PassiveGoalCreator(llm=llm)
goal: Goal = passive_goal_creator.run(query=query)

# Prompt Optimizerによるプロンプト最適化
prompt_optimizer = PromptOptimizer(llm=llm)
optimized_goal: OptimizedGoal = prompt_optimizer.run(query=goal.text)

# Response Optimizerによるレスポンス最適化
response_optimizer = ResponseOptimizer(llm=llm)
optimized_response: str = response_optimizer.run(query=optimized_goal.text)

pprint(f"Passive Goal Creator出力：{goal.text}")
print("\nプロンプト最適化結果：\n")
pprint(f"{optimized_goal.text}")
print("\nレスポンス最適化結果：\n")
pprint(f"{optimized_response}")

('Passive Goal '
 'Creator出力：カレーライスの作り方を調査し、ユーザーにわかりやすいレポートを生成する。具体的には、必要な材料、調理手順、調理時間、注意点などを含めた詳細なレポートを作成する。')

プロンプト最適化結果：

('インターネットを利用して、カレーライスの作り方に関する情報を調査し、必要な材料、調理手順、調理時間、注意点を含む詳細なレポートを作成する。レポートはユーザーが理解しやすい形式で、箇条書きや図を用いて視覚的に整理する。(測定基準: '
 'レポートに含まれる情報の正確性と詳細さ、ユーザーからのフィードバックの質、レポートの完成度を評価する。具体的には、必要な材料が全てリストアップされているか、調理手順がステップごとに明確に記載されているか、調理時間が具体的に示されているか、注意点が適切に強調されているかを確認する。さらに、ユーザーがレポートを理解しやすいと感じたかどうかをアンケートで評価する。)')

レスポンス最適化結果：

('目標分析:\n'
 'この目標は、カレーライスの作り方に関する詳細なレポートを作成することを目的としています。主要な要素は、必要な材料、調理手順、調理時間、注意点の4つです。レポートは視覚的に整理され、ユーザーが理解しやすい形式で提供される必要があります。情報の正確性と詳細さ、ユーザーからのフィードバック、レポートの完成度が評価基準となります。\n'
 '\n'
 'レスポンス仕様:\n'
 '- トーン: フレンドリーで親しみやすく、かつプロフェッショナル。\n'
 '- 構造: 材料リスト、調理手順、調理時間、注意点の順にセクションを分ける。箇条書きや図を用いて視覚的に整理。\n'
 '- 内容の焦点: 各セクションで情報を詳細かつ正確に提供し、特に注意点は強調する。\n'
 '\n'
 'AIエージェントへの指示:\n'
 '1. カレーライスの作り方に関する既存の情報をもとに、以下のセクションを作成してください。\n'
 '2. 必要な材料をリストアップし、分量を明記してください。\n'
 '3. 調理手順をステップごとに明確に記載してください。\n'
 '4. 調理時間を具体的に示してください。\n'
 '5. 調理中の注意点を強調して記載してください。\n'
 '6. 各セクシ

レスポンス最適化によって、AIエージェントに入力するクエリの最適化だけでなく、AIエージェントから出力される内容の最適化までを実現できる。  
